In [15]:
import numpy as np
import pandas as pd
path = u"./dataset/002.srt"
x = pd.read_csv(path, sep='\n')
y = np.array(x)

z = []
z.append(np.array(['1']))
for i in y:
    z.append(i)

for i in z:
    print(i)

s = ""
idx = 0
subPack = []
pack = []
for i in z:
    subPack.append(i[0])
    idx += 1
    if(idx % 3 == 0):
        pack.append(subPack)
        subPack = []
        idx = 0

start = []
end = []
for i in range(len(pack)):
    s = pack[i][1]
    s = s.replace(" --> ", "")
    s = s[0:8], s[12:20]
    a = list(s)
    pack[i][1] = a


['1']
['00:00:00,160 --> 00:00:02,510']
['核能發電到底是不是一個安全的技術呢']
['2']
['00:00:02,520 --> 00:00:03,470']
['在核災之後']
['3']
['00:00:03,480 --> 00:00:05,710']
['世界各國的核電發展又是如何呢']
['4']
['00:00:05,750 --> 00:00:07,510']
['廢核是國際趨勢嗎']
['5']
['00:00:07,610 --> 00:00:12,250']
['今天我們透過重啟核四公投來聊聊核電究竟安不安全嗎']
['6']
['00:00:23,550 --> 00:00:24,200']
['hello  大家好']
['7']
['00:00:24,210 --> 00:00:26,120']
['我是自己換回到2021公投']
['8']
['00:00:26,130 --> 00:00:26,530']
['只能哦']
['9']
['00:00:26,540 --> 00:00:27,940']
['這是這個系列的第4集']
['10']
['00:00:27,970 --> 00:00:31,000']
['那我們在前幾天已經談過公投要不要綁大選']
['11']
['00:00:31,010 --> 00:00:33,500']
['藻礁公投還有萊豬公投的3個議題']
['12']
['00:00:33,530 --> 00:00:34,800']
['那如果你還沒看完']
['13']
['00:00:34,830 --> 00:00:36,760']
['歡迎你有看完今天的影片之後呢']
['14']
['00:00:36,770 --> 00:00:38,920']
['回去看一看那今天這一集是第4集']
['15']
['00:00:38,930 --> 00:00:39,690']
['我們要來討論的']
['16']
['00:00:39,700 --> 00:00:41,880']
['是第十七案重啟核四公投']
['17']
['00:00:41,910 --> 00:00:44,820']
['不過我們發現這個議題比想像中還要複雜很多']
['18']
['00:00

In [16]:
import jieba
import jieba.analyse
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array



config_path = "./model/bert_base/chinese_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/bert_base/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/bert_base/chinese_L-12_H-768_A-12/vocab.txt'

model = build_transformer_model(
    config_path=config_path, checkpoint_path=checkpoint_path, with_nsp=True
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)

token = []
seg = []
token_ids = ""
segment_ids = ""
result = []

for i in range(len(pack)-1):
    if (len(pack[i][2])+len(pack[i+1][2])>=512):
        pack[i][2]=pack[i][2][-128:]

    token_ids, segment_ids = tokenizer.encode(
        pack[i][2], pack[i+1][2])
    
    token_ids, segment_ids = to_array([token_ids], [segment_ids])

    probas = model.predict([token_ids, segment_ids])[0]
    print(probas)
    result.append(probas)

# token_ids = np.array([[token_ids]])
# segment_ids = np.array([[segment_ids]])
# token_ids = token_ids.reshape(1,  1, -1)
# segment_ids = segment_ids.reshape(1,  1, -1)
# probas = model.predict([token_ids, segment_ids])[0]

for i in range(len(pack)-1):
    pack[i].append(result[i][0])

idxList = []
lessThanHalf = []
lessThanHalf.append('00:00:00')
for i in range(len(pack)-1):
    if(pack[i][3] < 0.2):
        idxList.append(i)
        lessThanHalf.append(pack[i][1][1])

combineText = []
j=''
j = pack[0][2]
for i in range(len(pack)-1):
    print(i)
    token_testId, Seg_testId = tokenizer.encode(pack[i][2], pack[i+1][2])
    token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
    result = model.predict([token_testId, Seg_testId])[0][0]

    # print(pack[i][2])
    if(result > 0.5):
        j += pack[i+1][2]
        if(i==len(pack)-2):
            j = pack[i+1][2]
            combineText.append(j)
            j=""
    else:
        j += pack[i+1][2]
        
        combineText.append(j)
        j = ""

wst=[]
for sentence in combineText:
    seq_list=jieba.lcut(sentence)
    wst.append(seq_list)

TF=np.zeros((np.shape(wst)[0],20))

combineStr=""
for i in combineText:
    combineStr+=i
Query=jieba.analyse.extract_tags(combineStr, topK=20, withWeight=False, allowPOS=())

for i in range(len(Query)):
    for j in range(len(wst)):
        for k in range(len(wst[j])):
            if(Query[i] == wst[j][k]):
                TF[j][i]= 1
                continue
print(TF)
keyWord=[]
deliver=[]
for i in TF:
    for j in range(len(i)):
        if(i[j]):
            deliver.append(Query[j])
    keyWord.append(deliver)
    deliver=[]

finalResult=[]
tmpProcess=[]
for i in range(len(combineText)):
    tmpProcess.append(lessThanHalf[i])
    #tmpProcess.append(lessThanHalf[i+1])
    tmpProcess.append(combineText[i])
    tmpProcess.append(keyWord[i])
    finalResult.append(tmpProcess)
    tmpProcess=[]

[9.9965560e-01 3.4436595e-04]
[9.9921393e-01 7.8600773e-04]
[9.9996066e-01 3.9357445e-05]
[9.9998975e-01 1.0285766e-05]
[0.85393566 0.14606437]
[0.9546738  0.04532614]
[0.8754484  0.12455161]
[0.02091188 0.9790881 ]
[0.00359487 0.9964051 ]
[9.9997723e-01 2.2815067e-05]
[0.8936845  0.10631551]
[9.9979478e-01 2.0528778e-04]
[9.997594e-01 2.406144e-04]
[0.99884474 0.00115523]
[0.996185   0.00381493]
[0.19087882 0.80912113]
[9.9995553e-01 4.4481385e-05]
[0.02798912 0.97201085]
[0.96975625 0.03024374]
[0.97287244 0.02712756]
[0.96907234 0.0309277 ]
[9.9997103e-01 2.9000796e-05]
[9.9995530e-01 4.4671993e-05]
[0.731792   0.26820806]
[0.98755205 0.01244798]
[9.9987340e-01 1.2657508e-04]
[0.99549603 0.00450393]
[0.9838382  0.01616181]
[0.9914352  0.00856476]
[0.09723452 0.9027655 ]
[0.99694735 0.00305265]
[0.88511896 0.11488099]
[0.9452024  0.05479756]
[0.9726518  0.02734818]
[0.99394846 0.00605153]
[0.99581796 0.00418203]
[0.9886568  0.01134319]
[0.05753742 0.9424626 ]
[9.9935144e-01 6.4852240

In [4]:
# token_ids, segment_ids = tokenizer.encode(
#     pack[0][2], pack[1][2])
# token_ids, segment_ids = to_array([token_ids], [segment_ids])

# probas = model.predict([token_ids, segment_ids])[0]

In [5]:
Query=jieba.analyse.extract_tags(combineStr, topK=20, withWeight=False, allowPOS=())

In [6]:
Query

['公投',
 '核電',
 '發展',
 '核四',
 '核能',
 '發電',
 '一個',
 '技術',
 '核災',
 '各國',
 '廢核',
 '國際',
 '趨勢',
 '罩門',
 '透過',
 '重啟',
 '玩嗎灣',
 '現況',
 '我們',
 '進入']

In [7]:
# for i in range(len(pack)-1):
#     if (len(pack[i][2])+len(pack[i+1][2])>=512):
#         pack[i][2]=pack[i][2][-128:]

#     token_ids, segment_ids = tokenizer.encode(
#         pack[i][2], pack[i+1][2])
    
#     token_ids, segment_ids = to_array([token_ids], [segment_ids])

#     probas = model.predict([token_ids, segment_ids])[0]
#     print(probas)
#     result.append(probas)


In [8]:
# token_ids = np.array([[token_ids]])
# segment_ids = np.array([[segment_ids]])
# token_ids = token_ids.reshape(1,  1, -1)
# segment_ids = segment_ids.reshape(1,  1, -1)



# probas = model.predict([token_ids, segment_ids])[0]


# token_ids, segment_ids = tokenizer.encode(
#     pack[0][2],
#     pack[1][2])
# token_ids, segment_ids = to_array([token_ids], [segment_ids])
# token_ids.shape
# probas = model.predict([token_ids, segment_ids])[0]
# probas



# for i in range(len(pack)-1):
#     pack[i].append(result[i][0])



# idxList = []
# lessThanHalf = []
# lessThanHalf.append('00:00:00')
# for i in range(1, len(pack)-1):
#     if(pack[i][3] < 0.2):
#         idxList.append(i)
#         lessThanHalf.append(pack[i][1][1])

# combineText = []

# S = pack[0][2]
# for i in range(len(pack)-1):

#     token_testId, Seg_testId = tokenizer.encode(pack[i][2], pack[i+1][2])
#     token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
#     result = model.predict([token_testId, Seg_testId])[0][0]
#     # print(pack[i][2])
#     if(result > 0.5):
#         S += pack[i+1][2]

#     else:
#         S += pack[i+1][2]
#         combineText.append(S)
#         S = ""

# wst=[]

# wst = ws(
#     combineText,
# )
# del ws


# for sentence in combineText:
#     seq_list=jieba.lcut(sentence)
#     wst.append(seq_list)

# seq_list = ws(
#     combineText,
#      sentence_segmentation = True, # To consider delimiters
#      segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
#     # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
#     # coerce_dictionary = dictionary2, # words in this dictionary are forced

# TF=np.zeros((np.shape(wst)[0],20))



# for i in range(len(Query)):
#     for j in range(len(wst)):
#         for k in range(len(wst[j])):
#             if(Query[i] == wst[j][k]):
#                 TF[j][i]= 1
#                 continue

# keyWord=[]
# deliver=[]
# for i in TF:
#     for j in range(len(i)):
#         if(i[j]):
#             deliver.append(Query[j])
#     keyWord.append(deliver)
#     deliver=[]
# finalResult=[]
# tmpProcess=[]
# time=[]

# for i in range(len(combineText)):
#     tmpProcess.append(lessThanHalf[i])
#     tmpProcess.append(lessThanHalf[i+1])
#     tmpProcess.append(combineText[i])
#     tmpProcess.append(keyWord[i])
#     finalResult.append(tmpProcess)
#     tmpProcess=[]

In [9]:
TF

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.]])

In [10]:
# S = combineText[0]
# combineText_1=[]
# for i in range(len(combineText)-1):

#     token_testId, Seg_testId = tokenizer.encode(combineText[i], combineText[i+1])
#     token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
#     result = (model.predict([token_testId, Seg_testId])[0][0])
#     print(result)
    # if(result > 0.01):
    #     S += combineText[i+1]

    # else:
    #     S += combineText[i+1]
    #     print(S)
    #     combineText_1.append(S)

In [11]:
for i in range(len(Query)):
    for j in range(len(wst)):
        for k in range(len(wst[j])):
            if(Query[i] == wst[j][k]):
                TF[j][i]= 1
                continue

In [25]:
wst

IndexError: list index out of range

In [13]:
# dataString = []
# for i in range(len(idxList)):
#     dataString.append(pack[i][2])


In [23]:
Str = []
a = ''
b = 0
k = 0
for i in range(len(idxList)):
    if(i == idxList[k]):
        for j in range(b, i+1):
            a += pack[j][2]
        Str.append(a)
        k += 1
        a = ''
        b = i+1


In [22]:
Str

[]

In [16]:
combineText

['核能發電到底是不是一個安全的技術呢？在核災之後，世界各國的核電發展又是如何呢？廢核是國際趨勢嗎？今天罩門透過重啟核四公投來聊聊核電究竟案板呈嗎？玩嗎灣的核能發展現況，那明天我們就會正式進入這次核四公投的正反意見討論，當然照慣例，有在開來之前還是要先進了工商服務時間，台灣第一，場專門為約定影音創作者舉辦年會，一趟即將登場。嘍不然你是剛踏入影劇圈，還在努力殺出重圍佈置已經有點成果正在思考變現或其他問題上，']

In [21]:
combineStr="".join(combineText)

In [20]:
combineStr

'核能發電到底是不是一個安全的技術呢？在核災之後，世界各國的核電發展又是如何呢？廢核是國際趨勢嗎？今天罩門透過重啟核四公投來聊聊核電究竟案板呈嗎？玩嗎灣的核能發展現況，那明天我們就會正式進入這次核四公投的正反意見討論，當然照慣例，有在開來之前還是要先進了工商服務時間，台灣第一，場專門為約定影音創作者舉辦年會，一趟即將登場。嘍不然你是剛踏入影劇圈，還在努力殺出重圍佈置已經有點成果正在思考變現或其他問題上，'

In [28]:
finalResult[0]

['00:00:00',
 '00:00:17',
 '核能發電到底是不是一個安全的技術呢？在核災之後，世界各國的核電發展又是如何呢？廢核是國際趨勢嗎？今天罩門透過重啟核四公投來聊聊核電究竟案板呈嗎？玩嗎灣的核能發展現況，那明天我們就會正式進入這次核四公投的正反意見討論，當然照慣例，有在開來之前還是要先進了工商服務時間，台灣第一，場專門為約定影音創作者舉辦年會，一趟即將登場。嘍不然你是剛踏入影劇圈，還在努力殺出重圍佈置已經有點成果正在思考變現或其他問題上，',
 ['公投',
  '核電',
  '發展',
  '核四',
  '核能',
  '發電',
  '一個',
  '技術',
  '核災',
  '各國',
  '廢核',
  '國際',
  '趨勢',
  '罩門',
  '透過',
  '重啟',
  '玩嗎灣',
  '現況',
  '我們',
  '進入']]